In [7]:
import numpy as np
import networkx as nx
from qiskit import transpile, QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import UnitaryGate
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer, AerSimulator

In [8]:
def F1(qc, x1, x2, sol):
    qc.barrier()
    qc.mcx([x1[0], x2[0]], sol)
    qc.barrier()
    qc.x(x2)
    qc.mcx([x1[1], x2[0], x2[1]], sol)
    qc.x(x2)
    qc.barrier()
    qc.x(x1)
    qc.mcx([x2[1], x1[1], x1[0]], sol)
    qc.x(x1)
    qc.barrier()
    
def F2(qc, x1, x2, sol):
    qc.barrier()
    qc.mcx([x1[1], x2[1]], sol)
    qc.barrier()
    qc.x(x2)
    qc.mcx([x1[0], x2[1], x2[0]], sol)
    qc.x(x2)
    qc.barrier()
    qc.x(x1)
    qc.mcx([x2[0], x1[1], x1[0]], sol)
    qc.x(x1)
    qc.barrier()


In [9]:
def Adder(qc, x1, x2, sol):
    F1(qc, x1, x2, sol[1])
    F2(qc, x1, x2, sol[0])

In [10]:
def get_phase_gate(n):
    size = 2**n
    i = -np.eye(size)
    i[0][0] = 1
    phase_gate = UnitaryGate(i)
    return phase_gate

In [11]:
def diffuser(n):
    qc = QuantumCircuit(n)
    for q in range(n):
        qc.h(q)
    qc.append(get_phase_gate(n), range(n))
    for q in range(n):
        qc.h(q)
    U_s = qc.to_gate()
    U_s.name = "U_s"
    return U_s

In [12]:
def lights_out_oracle(qc, x1, x2, x3, a, y, y1, y2, y3, y4, sol):
    """
    A = [1, 1, 0; 1, 1, 1; 0, 1, 1]
    i = [2, 0, 0]
    x = [0, 1, 2] => 100100
    """
    # x1 + x2 + 2 = 0
    Adder(qc, x1, x2, y)
    Adder(qc, y, a, y1)
    
    # x1 + x2 + x3 = 0
    Adder(qc, x1, x2, y2)
    Adder(qc, y2, x3, y3)
    
    # x2 + x3 = 0
    Adder(qc, x2, x3, y4)
    
    qc.x(y1)
    qc.x(y3)
    qc.x(y4)
    
    qc.mcx([y1[0], y1[1], y4[0], y4[1], y3[0], y3[1]], sol)
    
    # x1 + x2 + 2 = 0
    Adder(qc, x1, x2, y)
    Adder(qc, y, a, y1)
    
    # x1 + x2 + x3 = 0
    Adder(qc, x1, x2, y2)
    Adder(qc, y2, x3, y3)
    
    # x2 + x3 = 0
    Adder(qc, x2, x3, y4)
    
    qc.x(y1)
    qc.x(y3)
    qc.x(y4)
    
    qc.barrier()
    
    qc.append(diffuser(6), range(6))

In [13]:
def lights_out_solver():
    
    x1 = QuantumRegister(2, name="x1")
    x2 = QuantumRegister(2, name="x2")
    x3 = QuantumRegister(2, name="x3")
    a = QuantumRegister(2, name="a")
    y = QuantumRegister(2, name="y")
    y1 = QuantumRegister(2, name="y1")
    y2 = QuantumRegister(2, name="y2")
    y3 = QuantumRegister(2, name="y3")
    y4 = QuantumRegister(2, name="y4")
    sol = QuantumRegister(1, name="sol")
    cbit1 = ClassicalRegister(2, name="cbit1")
    cbit2 = ClassicalRegister(2, name="cbit2")
    cbit3 = ClassicalRegister(2, name="cbit3")
    
    qc = QuantumCircuit(x1, x2, x3, a, y, y1, y2, y3, y4, sol, cbit1, cbit2, cbit3)
    
    # INITIALIZATION
    # - STATE on SOLUTION QUBIT
    qc.x(sol)
    qc.h(sol)
    
    # SUPERPOSITION OF BITSTRING
    qc.initialize([1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3), 0], x1)
    qc.initialize([1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3), 0], x2)
    qc.initialize([1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3), 0], x3)
    
    
    # Initial state to solve
    qc.x(a[1])
    
    lights_out_oracle(qc, x1, x2, x3, a, y, y1, y2, y3, y4, sol)
    lights_out_oracle(qc, x1, x2, x3, a, y, y1, y2, y3, y4, sol)
    lights_out_oracle(qc, x1, x2, x3, a, y, y1, y2, y3, y4, sol)
    lights_out_oracle(qc, x1, x2, x3, a, y, y1, y2, y3, y4, sol)
    
    qc.measure(x1, cbit1)
    qc.measure(x2, cbit2)
    qc.measure(x3, cbit3)
    
    return qc

In [15]:
qc = lights_out_solver()
qc.draw()

┌────────────────────────────────────────┐      ░       ░           »
   x1_0: ┤0                                       ├──────░───■───░───────────»
         │  Initialize(0.57735,0.57735,0.57735,0) │      ░   │   ░           »
   x1_1: ┤1                                       ├──────░───┼───░────────■──»
         ├────────────────────────────────────────┤      ░   │   ░ ┌───┐  │  »
   x2_0: ┤0                                       ├──────░───■───░─┤ X ├──■──»
         │  Initialize(0.57735,0.57735,0.57735,0) │      ░   │   ░ ├───┤  │  »
   x2_1: ┤1                                       ├──────░───┼───░─┤ X ├──■──»
         ├────────────────────────────────────────┤      ░   │   ░ └───┘  │  »
   x3_0: ┤0                                       ├──────░───┼───░────────┼──»
         │  Initialize(0.57735,0.57735,0.57735,0) │      ░   │   ░        │  »
   x3_1: ┤1                                       ├──────░───┼───░────────┼──»
         └────────────────────────────────────────┘      ░   │   ░        │  »
    a_0: ────────────────────────────────────────────────░───┼───░────────┼──»
                           ┌───┐                         ░   │   ░        │  »
    a_1: ──────────────────┤ X ├─────────────────────────░───┼───░────────┼──»
                           └───┘                         ░   │   ░        │  »
    y_0: ────────────────────────────────────────────────░───┼───░────────┼──»
                                                         ░ ┌─┴─┐ ░      ┌─┴─┐»
    y_1: ────────────────────────────────────────────────░─┤ X ├─░──────┤ X ├»
                                                         ░ └───┘ ░      └───┘»
   y1_0: ────────────────────────────────────────────────░───────░───────────»
                                                         ░       ░           »
   y1_1: ────────────────────────────────────────────────░───────░───────────»
                                                         ░       ░           »
   y2_0: ────────────────────────────────────────────────░───────░───────────»
                                                         ░       ░           »
   y2_1: ────────────────────────────────────────────────░───────░───────────»
                                                         ░       ░           »
   y3_0: ────────────────────────────────────────────────░───────░───────────»
                                                         ░       ░           »
   y3_1: ────────────────────────────────────────────────░───────░───────────»
                                                         ░       ░           »
   y4_0: ────────────────────────────────────────────────░───────░───────────»
                                                         ░       ░           »
   y4_1: ────────────────────────────────────────────────░───────░───────────»
                           ┌───┐                   ┌───┐ ░       ░           »
    sol: ──────────────────┤ X ├───────────────────┤ H ├─░───────░───────────»
                           └───┘                   └───┘ ░       ░           »
cbit1: 2/════════════════════════════════════════════════════════════════════»
                                                                             »
cbit2: 2/════════════════════════════════════════════════════════════════════»
                                                                             »
cbit3: 2/════════════════════════════════════════════════════════════════════»
                                                                             »
«               ░ ┌───┐     ┌───┐ ░  ░       ░                 ░ ┌───┐     »
«   x1_0: ──────░─┤ X ├──■──┤ X ├─░──░───────░────────■────────░─┤ X ├──■──»
«               ░ ├───┤  │  ├───┤ ░  ░       ░        │        ░ ├───┤  │  »
«   x1_1: ──────░─┤ X ├──■──┤ X ├─░──░───■───░────────┼────────░─┤ X ├──■──»
«         ┌───┐ ░ └───┘  │  └───┘ ░  ░   │   ░ ┌───┐  │  ┌───┐ ░ └───┘  │  »
«   x2_0: ┤ X ├─░────────┼────────░──░───┼───░─┤ X ├──■──┤ X ├─░─────

In [16]:
qasm_simulator = Aer.get_backend('qasm_simulator')
transpiled_qc = transpile(qc, qasm_simulator)
result = qasm_simulator.run(transpiled_qc, shots=1024).result()
plot_histogram(result.get_counts(), figsize=(20, 20), filename="./3_node_Z3_sol.png")